In [1]:
include("./torontonian.jl");
include("./tor_tests.jl"); #this line also runs all the tests in the file

Test Summary:                                                         | Pass  Total
Torontonian tests                                                     |  615    615
  Torontonian (from definition) vs. analytical result                 |  204    204
  Torontonian (recursive) vs. analytical result                       |  204    204
  Torontonian (from definition) vs. Torontonian (recursive)           |  204    204
  Torontonian of two-mode squeezed vacuum with mean photon number 1.0 |    3      3
Test Summary:          | Pass  Total
probabilities sum to 1 |   18     18
Test Summary:                           | Pass  Total
probabilities are equal in both methods | 2044   2044


In [2]:
#checking that the Torontonian function (from definition) gives the correct analytical results
@testset "Torontonian (from definition) vs. analytical result" begin
    for i = 2:1:13
        for k = 1.1:0.37:7.3
            @test isapprox(analytical_tor(i, k), tor(gen_omats(i, k)))
        end
    end
end;

Test Summary:                                       | Pass  Total
Torontonian (from definition) vs. analytical result |  204    204


In [3]:
#checking that the Torontonian function (recursive) gives the correct analytical results
@testset "Torontonian (recursive) vs. analytical result" begin
    for i = 2:1:13
        for k = 1.1:0.37:7.3
            @test isapprox(analytical_tor(i, k), rec_tor(gen_omats(i, k)))
        end
    end
end;

Test Summary:                                 | Pass  Total
Torontonian (recursive) vs. analytical result |  204    204


In [4]:
#Computing threshold detection probabilities
N = 4 #number of modes
cov = random_covariance(N) #rendom covariance matrix
println("pattern", " | ", "prob (definition)", " | ", "prob (recursive)")
for p in product([[0 1] for i = 1:N]...) #this line generates all possible detection patterns
    pattern = collect(p)
    prob_df = threshold_detection_prob(cov, pattern, recursive = false)
    prob_rc = threshold_detection_prob(cov, pattern, recursive = true)
    println(pattern, " | ", real(prob_df), " | ", real(prob_rc))
end

pattern | prob (definition) | prob (recursive)
[0, 0, 0, 0] | 0.049396549671586736 | 0.049396549671586736
[1, 0, 0, 0] | 0.029665100576935374 | 0.029665100576935374
[0, 1, 0, 0] | 0.01755499244057341 | 0.01755499244057341
[1, 1, 0, 0] | 0.027642719511483407 | 0.027642719511483407
[0, 0, 1, 0] | 0.026959656172359132 | 0.026959656172359132
[1, 0, 1, 0] | 0.025314970564479112 | 0.025314970564479112
[0, 1, 1, 0] | 0.02478234548550178 | 0.02478234548550178
[1, 1, 1, 0] | 0.03597949936677541 | 0.03597949936677541
[0, 0, 0, 1] | 0.04254968966897921 | 0.04254968966897921
[1, 0, 0, 1] | 0.028503822740049065 | 0.028503822740049065
[0, 1, 0, 1] | 0.0957099548509343 | 0.09570995485093425
[1, 1, 0, 1] | 0.15006597614034503 | 0.1500659761403449
[0, 0, 1, 1] | 0.04194334297936686 | 0.04194334297936685
[1, 0, 1, 1] | 0.0418134383534186 | 0.04181343835341859
[0, 1, 1, 1] | 0.11252084633520647 | 0.11252084633520643
[1, 1, 1, 1] | 0.24959709514200612 | 0.24959709514200584
